### Imports

In [1]:
import torch
import rasterio
import numpy as np
import cv2

### Preprocessing

In [15]:
def preProcess1(img):
    img = np.clip(img, -20, 5)
    img = 10 ** (img / 20.0)
    return np.clip(img, 0, 255)

def preProcess2(img):
    


### Spatial augmentation

In [31]:
# TODO: give limits to random
def spatialAug1(sentinel1):
    vv = np.array(sentinel1.read(1), dtype=float)
    vh = np.array(sentinel1.read(2), dtype=float)

    i1 = np.random.randint(low=0, high=vv.shape[0]-2, size=None)
    i2 = np.random.randint(low=i1, high=vv.shape[0]-1, size=None)
    j1 = np.random.randint(low=0, high=vv.shape[1]-2, size=None)
    j2 = np.random.randint(low=j1, high=vv.shape[1]-1, size=None)

    vv_croped = vv[i1:i2, j1:j2]
    vh_croped = vh[i1:i2, j1:j2]

    vv_resized = cv2.resize(vv_croped, dsize=(512, 512))
    vh_resized = cv2.resize(vh_croped, dsize=(512, 512))
    fill = np.zeros((512,512))

    return torch.from_numpy(np.stack((vv_resized, vh_resized, fill), axis=2))

def spatialAug2(sentinel2):
    r = np.array(sentinel2.read(3), dtype=float)
    g = np.array(sentinel2.read(2), dtype=float)
    b = np.array(sentinel2.read(1), dtype=float)

    i1 = np.random.randint(low=0, high=r.shape[0]-2, size=None)
    i2 = np.random.randint(low=i1, high=r.shape[0]-1, size=None)
    j1 = np.random.randint(low=0, high=r.shape[1]-2, size=None)
    j2 = np.random.randint(low=j1, high=r.shape[1]-1, size=None)

    r_croped = r[i1:i2, j1:j2]
    g_croped = g[i1:i2, j1:j2]
    b_croped = b[i1:i2, j1:j2]

    r_resized = cv2.resize(r_croped, dsize=(512, 512))
    g_resized = cv2.resize(g_croped, dsize=(512, 512))
    b_resized = cv2.resize(b_croped, dsize=(512, 512))

    return torch.from_numpy(np.stack((r_resized, g_resized, b_resized), axis=2))

### Models

In [4]:
resnet1  = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
resnet2  = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)

Using cache found in /home/charlie/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/charlie/.cache/torch/hub/pytorch_vision_v0.10.0


### Model training

In [34]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def criterion(x, y):
    sim = torch.nn.CosineSimilarity()
    l = np.empty(x.shape)

    for i in range(len(l)):
        lxyi = torch.exp(sim(x[i], y[i]))
        lyxi = torch.exp(sim(y[i], x[i]))

        simxy, simxx, simyx, simyy = 0,0,0,0
        for j in range(len(x)):
            simxy += torch.exp(sim(x[i], y[j]))
            simyx += torch.exp(sim(y[i], x[j]))
            if j!=i:
                simxx += torch.exp(sim(x[i], x[j]))
                simyy += torch.exp(sim(y[i], y[j]))
               

        l[i] = -torch.log(lxyi/(simxx + simxy)) + -torch.log(lyxi/(simyy + simyx))
    return torch.sum(l)

def preTrain(model, batch, weight_decay=0.0, optimizer="sgd", learning_rate=0.1, momentum=0.9, num_epochs=10):
    assert optimizer in ("sgd", "adam")
    if optimizer == "sgd":
        optimizer1 = torch.optim.SGD(model[0].parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
        optimizer2 = torch.optim.SGD(model[1].parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
    else:
        optimizer1 = torch.optim.Adam(model[0].parameters(), lr=learning_rate, weight_decay=weight_decay)
        optimizer2 = torch.optim.Adam(model[1].parameters(), lr=learning_rate, weight_decay=weight_decay)

    iters, iters_acc, losses, train_acc, val_acc = [], [], [], [], []
    # training
    n = 0
    for epoch in range(num_epochs):
        for imgs in batch:
            imgs1 = imgs[0].to(device)
            imgs2 = imgs[1].to(device)
            model[0].train()
            model[1].train()
            x = model[0](imgs1)
            y = model[1](imgs2)
            loss = criterion(x, y)
            loss.backward()
            optimizer1.step()
            optimizer1.zero_grad()
            optimizer2.step()
            optimizer2.zero_grad()

            iters.append(n)
            losses.append(float(loss)/len(batch))

            if n % 20 == 0 :
            #   train_acc.append(get_accuracy(model, train=True)) # training accuracy
            #   val_acc.append(get_accuracy(model, train=False))  # test accuracy
              iters_acc.append(n)

            n += 1

In [35]:
# Open the GeoTIFF image
sentinel1_image_path = "sentinel1_example.tif"
sentinel2_image_path = "sentinel2_example.tif"

sentinel1 = rasterio.open(sentinel1_image_path)
sentinel2 = rasterio.open(sentinel2_image_path)

img1 = spatialAug1(sentinel1)
img2 = spatialAug2(sentinel2)

print(img1.shape, img2.shape)

preTrain((resnet1, resnet2), (img1, img2))

# preProcess1(img1)

torch.Size([512, 512, 3]) torch.Size([512, 512, 3])


RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [512, 3]

### Model evaluation